FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
###query 1 RDD

import csv,time
from io import StringIO
from pyspark.sql import SparkSession


sc = SparkSession \
    .builder \
    .appName("Query 1 RDD") \
    .config("spark.executor.instances", "4") \
    .getOrCreate() \
    .sparkContext


def parse_csv_line(line):
    # Use StringIO to treat the line as a file-like object
    f = StringIO(line)
    # Use csv.reader to correctly parse the CSV line
    reader = csv.reader(f)
    return next(reader) 
def help1(data):
    try:
        age=int(data)
        if age<18 and age>0:
            return "child"
        if age<25:
            return "young adult"
        if age<65 :
            return "adult"
        if age>64:
            return "old"
        else:
            return "no individual victim"
    except:
        return "error"
    

start_time = time.time()
    
rdd1  = sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv")\
.map(parse_csv_line)

rdd2= sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv")\
.map(parse_csv_line)

header1 = rdd1.first()
header2 = rdd2.first()

# Filter out the header
rdd1_data = rdd1.filter(lambda line: line != header1)
rdd2_data = rdd2.filter(lambda line: line != header2)

crime_data = rdd1_data.union(rdd2_data) \
.filter(lambda pair: pair[9].find("AGGRAVATED") != -1 ) \
.map(lambda data: (help1(data[11]),1)) \
.reduceByKey(lambda a, b: a + b) \
.sortBy( lambda pair : pair[1], ascending=False )

print(crime_data.collect())

end_time = time.time()

print(f"Time taken: {end_time-start_time:.2f} seconds")



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('adult', 121093), ('young adult', 38703), ('child', 10830), ('old', 5985)]
Time taken: 28.52 seconds

In [22]:
####query 1 dataframe

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, TimestampType
from pyspark.sql.functions import udf, col
from pyspark.sql import functions as F
import time


spark = SparkSession \
    .builder \
    .appName("Query 1 Dataframe") \
    .config("spark.executor.instances", "4") \
    .getOrCreate() 


def age_group(age_str):
    try:
        age=int(age_str)
        if age<18 and age>0:
            return "child"
        if age<25:
            return "young adult"
        if age<65 :
            return "adult"
        if age>64:
            return "old"
        else:
            return "no individual victim"
    except:
        return "error"


start_time=time.time()
dataframe1= spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=True)
dataframe2= spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv",header=True)

age_udf=udf(age_group,StringType())
dataframe=dataframe1.union(dataframe2)\
.filter(col("Crm Cd Desc").contains("AGGRAVATED"))\
.withColumn("age_group",age_udf(col("Vict Age")))\
.groupBy("age_group").agg(F.count("*").alias("count"))\
.orderBy("count",ascending=False)


dataframe.show()


end_time = time.time()

print(f"Time taken: {end_time-start_time:.2f} seconds")



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------+
|  age_group| count|
+-----------+------+
|      adult|121093|
|young adult| 38703|
|      child| 10830|
|        old|  5985|
+-----------+------+

Time taken: 2.37 seconds

In [7]:
####query2 dataframe

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, TimestampType
from pyspark.sql.functions import udf, col,count,when
from pyspark.sql import functions as F
from pyspark.sql.window import Window

import time


spark = SparkSession \
    .builder \
    .appName("Query 2 Dataframe") \
    .getOrCreate() 


window_spec = Window.partitionBy("Year").orderBy(F.desc("closed_case_rate"))


start_time=time.time()


dataframe1= spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=True)
dataframe2= spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv",header=True)



dataframe=dataframe1.union(dataframe2)\
.withColumn("year",col("Date Rptd").substr(7,4))\
.select("year","AREA NAME","Status")\
.groupBy("year","AREA NAME").agg((count(when(col("Status") != "IC", 1)) / count("*") ).alias("closed_case_rate"))\
.withColumn("#", F.row_number().over(window_spec) )\
.filter(col("#") <= 3)\
.withColumnRenamed("AREA NAME", "precinct")

dataframe.show(24)

end_time = time.time()

print(f"Time taken: {end_time-start_time:.2f} seconds")


    




FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+-------------------+---+
|year|   precinct|   closed_case_rate|  #|
+----+-----------+-------------------+---+
|2010|    Rampart|0.32947355855318133|  1|
|2010|    Olympic|0.31962706191728424|  2|
|2010|     Harbor| 0.2963203463203463|  3|
|2011|    Olympic|0.35212167689161555|  1|
|2011|    Rampart|0.32511779630300836|  2|
|2011|     Harbor| 0.2865220520201501|  3|
|2012|    Olympic| 0.3441481831052383|  1|
|2012|    Rampart|  0.329464181029429|  2|
|2012|     Harbor| 0.2981513327601032|  3|
|2013|    Olympic| 0.3352812271731191|  1|
|2013|    Rampart| 0.3208287360549221|  2|
|2013|     Harbor| 0.2916422459266206|  3|
|2014|   Van Nuys| 0.3180567315834039|  1|
|2014|West Valley| 0.3131198995605775|  2|
|2014|    Mission| 0.3116279069767442|  3|
|2015|   Van Nuys| 0.3264134698172773|  1|
|2015|West Valley| 0.3027597402597403|  2|
|2015|    Mission|0.30179460678380154|  3|
|2016|   Van Nuys|0.31880755720117726|  1|
|2016|West Valley| 0.3154798761609907|  2|
|2016|   Fo

In [4]:
###query2 spark sql api


from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, TimestampType
from pyspark.sql.functions import udf, col,count,when
from pyspark.sql import functions as F
from pyspark.sql.window import Window

import time


spark = SparkSession \
    .builder \
    .appName("Query 2 SQL API") \
    .getOrCreate()


start_time=time.time()

dataframe1= spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=True)
dataframe2= spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv",header=True)

dataframe=dataframe1.union(dataframe2)

dataframe.createOrReplaceTempView("Dataset")
query= """
    WITH extracted_data AS (
        SELECT 
            substr(`Date Rptd`, 7, 4) AS year,
            `AREA NAME` AS precinct,
            Status
        FROM Dataset
    ),
    aggregated_data AS (
        SELECT
            year,
            precinct,
            COUNT(CASE WHEN Status != 'IC' THEN 1 END)  / COUNT(*) AS closed_case_rate
        FROM extracted_data
        GROUP BY year, precinct
    ),
    ranked_data AS (
        SELECT
            year,
            precinct,
            closed_case_rate,
            ROW_NUMBER() OVER (PARTITION BY year ORDER BY closed_case_rate DESC) AS `#`
        FROM aggregated_data
    )
    SELECT 
        year,
        precinct,
        closed_case_rate,
        `#`
    FROM ranked_data
    WHERE `#` <= 3
"""
res=spark.sql(query)
res.show(24)

end_time = time.time()

print(f"Time taken: {end_time-start_time:.2f} seconds")



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+-------------------+---+
|year|   precinct|   closed_case_rate|  #|
+----+-----------+-------------------+---+
|2010|    Rampart|0.32947355855318133|  1|
|2010|    Olympic|0.31962706191728424|  2|
|2010|     Harbor| 0.2963203463203463|  3|
|2011|    Olympic|0.35212167689161555|  1|
|2011|    Rampart|0.32511779630300836|  2|
|2011|     Harbor| 0.2865220520201501|  3|
|2012|    Olympic| 0.3441481831052383|  1|
|2012|    Rampart|  0.329464181029429|  2|
|2012|     Harbor| 0.2981513327601032|  3|
|2013|    Olympic| 0.3352812271731191|  1|
|2013|    Rampart| 0.3208287360549221|  2|
|2013|     Harbor| 0.2916422459266206|  3|
|2014|   Van Nuys| 0.3180567315834039|  1|
|2014|West Valley| 0.3131198995605775|  2|
|2014|    Mission| 0.3116279069767442|  3|
|2015|   Van Nuys| 0.3264134698172773|  1|
|2015|West Valley| 0.3027597402597403|  2|
|2015|    Mission|0.30179460678380154|  3|
|2016|   Van Nuys|0.31880755720117726|  1|
|2016|West Valley| 0.3154798761609907|  2|
|2016|   Fo

In [9]:
####2b
####make parquet dataset


from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, TimestampType
from pyspark.sql.functions import udf, col,count,when
from pyspark.sql import functions as F
from pyspark.sql.window import Window

import time


spark = SparkSession \
    .builder \
    .appName("Query 2b write parquet") \
    .getOrCreate()

dataframe1= spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=True)
dataframe2= spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv",header=True)
dataframe=dataframe1.union(dataframe2)

dataframe.coalesce(1).write.mode("overwrite").parquet("s3://groups-bucket-dblab-905418150721/group35/main_dataset_parquet") ##coalesce gia 1 file






FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
####2b test parquet file on 2a query dataframe

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, TimestampType
from pyspark.sql.functions import udf, col,count,when
from pyspark.sql import functions as F
from pyspark.sql.window import Window

import time


spark = SparkSession \
    .builder \
    .appName("Query 2b test parquet for 2b Dataframe") \
    .getOrCreate()

start_time=time.time()

dataframe = spark.read.parquet("s3://groups-bucket-dblab-905418150721/group35/main_dataset_parquet")

dataframe=dataframe.withColumn("year",col("Date Rptd").substr(7,4))\
.select("year","AREA NAME","Status")\
.groupBy("year","AREA NAME").agg((count(when(col("Status") != "IC", 1)) / count("*") ).alias("closed_case_rate"))\
.withColumn("#", F.row_number().over(window_spec) )\
.filter(col("#") <= 3)\
.withColumnRenamed("AREA NAME", "precinct")

dataframe.show(60)

end_time = time.time()

print(f"Time taken: {end_time-start_time:.2f} seconds")



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+-------------------+---+
|year|   precinct|   closed_case_rate|  #|
+----+-----------+-------------------+---+
|2010|    Rampart|0.32947355855318133|  1|
|2010|    Olympic|0.31962706191728424|  2|
|2010|     Harbor| 0.2963203463203463|  3|
|2011|    Olympic|0.35212167689161555|  1|
|2011|    Rampart|0.32511779630300836|  2|
|2011|     Harbor| 0.2865220520201501|  3|
|2012|    Olympic| 0.3441481831052383|  1|
|2012|    Rampart|  0.329464181029429|  2|
|2012|     Harbor| 0.2981513327601032|  3|
|2013|    Olympic| 0.3352812271731191|  1|
|2013|    Rampart| 0.3208287360549221|  2|
|2013|     Harbor| 0.2916422459266206|  3|
|2014|   Van Nuys| 0.3180567315834039|  1|
|2014|West Valley| 0.3131198995605775|  2|
|2014|    Mission| 0.3116279069767442|  3|
|2015|   Van Nuys| 0.3264134698172773|  1|
|2015|West Valley| 0.3027597402597403|  2|
|2015|    Mission|0.30179460678380154|  3|
|2016|   Van Nuys|0.31880755720117726|  1|
|2016|West Valley| 0.3154798761609907|  2|
|2016|   Fo

In [10]:
####query 3

from sedona.spark import *
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Create spark Session
spark = SparkSession.builder \
    .appName("GeoJSON read") \
    .getOrCreate()

# Create sedona context
sedona = SedonaContext.create(spark)
# Read the file from s3
geojson_path = "s3://initial-notebook-data-bucket-dblab-905418150721/2010_Census_Blocks.geojson"
blocks_df = sedona.read.format("geojson") \
            .option("multiLine", "true").load(geojson_path) \
            .selectExpr("explode(features) as features") \
            .select("features.*")
# Formatting magic
blocks_census = blocks_df.select( \
                [col(f"properties.{col_name}").alias(col_name) for col_name in \
                blocks_df.schema["properties"].dataType.fieldNames()] + ["geometry"]) \
            .drop("properties") \
            .drop("type")
census=blocks_census.select("COMM","POP_2010","ZCTA10").na.fill({"POP_2010": 0}).groupBy("COMM","ZCTA10").agg(F.sum("POP_2010").alias("POP_2010"))


income= spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/LA_income_2015.csv", header=True)

res1= income.withColumn( "Estimated Median Income", F.regexp_replace(col("Estimated Median Income"), "[$,]", "").cast("float"))\
.join(census,census["ZCTA10"]==income["Zip Code"])\
.withColumn("total_income",col("Estimated Median Income")*col("POP_2010") )\
.groupBy("COMM").agg(
                    F.sum("POP_2010").alias("total_population"),
                    F.sum("total_income").alias("comm_total_income"))\
.withColumn("average_income", col("comm_total_income")/col("total_population"))


dataframe1= spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=True)
dataframe2= spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv",header=True)
dataframe=dataframe1.union(dataframe2)

res2=


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------------+-----------------+-----------------+
|                COMM|total_population|comm_total_income|   average_income|
+--------------------+----------------+-----------------+-----------------+
|         Culver City|           38883|    2.905977048E9|74736.44132397191|
|     North Lancaster|            1101|       4.438881E7|40316.81198910082|
|Rosewood/East Gar...|            1164|      6.2048184E7|          53306.0|
+--------------------+----------------+-----------------+-----------------+
only showing top 3 rows

In [32]:
# # # # income= spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/LA_income_2015.csv", header=True)

# # # # income.filter(col("Community").contains("Antelope")).show(3)

# res=income.join(blocks_census,blocks_census["ZCTA10"]==income["Zip Code"])
# print(res.select("ZCTA10").distinct().count())

# # print(res.select("Community").distinct().count())
# # # res.show(3)
# # # print(res.count())
# # # print(income.select("Community").distinct().count())
# # # print(census.select("COMM").distinct().count())
# print(blocks_census.select("COMM").distinct().count())
# print(blocks_census.select("ZCTA10").distinct().count())

# blocks_census.show(3)
census.show(3)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------------+
|ZCTA10|sum(POP_2010)|
+------+-------------+
| 90807|        31481|
| 91326|        33708|
| 90094|         5464|
+------+-------------+
only showing top 3 rows

In [13]:
income.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+-----------------------+
|Zip Code|           Community|Estimated Median Income|
+--------+--------------------+-----------------------+
|   90001|Los Angeles (Sout...|                $33,887|
|   90002|Los Angeles (Sout...|                $30,413|
|   90003|Los Angeles (Sout...|                $30,805|
|   90004|Los Angeles (Hanc...|                $40,612|
|   90005|Los Angeles (Hanc...|                $31,142|
|   90006|Los Angeles (Byza...|                $31,521|
|   90007|Los Angeles (Sout...|                $22,304|
|   90008|Los Angeles (Bald...|                $36,564|
|   90010|Los Angeles (Hanc...|                $45,786|
|   90011|Los Angeles (Sout...|                $30,251|
+--------+--------------------+-----------------------+
only showing top 10 rows

In [17]:
res.show(50)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------------+-----------------+------------------+
|                COMM|total_population|comm_total_income|    average_income|
+--------------------+----------------+-----------------+------------------+
|         Culver City|           38883|    2.905977048E9| 74736.44132397191|
|     North Lancaster|            1101|       4.438881E7| 40316.81198910082|
|Rosewood/East Gar...|            1164|      6.2048184E7|           53306.0|
|East Rancho Domin...|           15135|     6.35064576E8|41959.998414271555|
|      Toluca Terrace|            1301|        6.30972E7| 48499.00076863951|
|        Elysian Park|            5267|     1.92298904E8| 36510.13935826846|
|            Longwood|            4210|     1.61369296E8| 38329.99904988123|
|         Pico Rivera|           62942|    3.509822933E9| 55762.81231927806|
|              Malibu|           12645|    1.563922097E9|123679.09031237643|
|       Green Meadows|           19821|      6.0605866E8| 30576.59351193179|